In [16]:
import pandas as pd
import os
from paths import *

In [20]:
guide_file = pd.read_csv(os.path.join(src_, "guide_train.csv"))
guide_file = guide_file[~guide_file["segment_nostress"].isin(["sil", "sp", "spn"])]
guide_file = guide_file[guide_file['nSample'] > 400]
guide_file = guide_file[guide_file['nSample'] <= 8000]

In [21]:
len(guide_file["word_path"].unique().tolist()) // 128

6055

In [22]:
len(guide_file["phone_path"].unique().tolist()) // 128

21629